In [1]:
import pandas as pd
import numpy as np
from collections import Counter

import pickle
from flask import Flask, render_template, request

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

clean_df2 = pd.read_csv('/Users/yinglin/Desktop/pickle 1/clean.csv')
clean_df2 

import sklearn as skl

target_column = "Diabetes"
target_column_onehot = 'yes'

# features
columns=[
         'Categorical_BMI',
         'Education',
         'Poverty_Ratio',
         'Race',
         'Age'
         ]
         
columnsCAT=[
         'Categorical_BMI',
         'Education',
         'Race'
         ]

columnsNUM=[
         'Poverty_Ratio',
         'Age'
         ]

XYcolumns = columns+[target_column]
print(XYcolumns)
XY = clean_df2.loc[ :, XYcolumns ].copy()
XY = XY.dropna()

XY[columnsCAT].astype('str')
XY[columnsNUM].astype('float')
X = pd.get_dummies( XY[ columns ], columns=columnsCAT )

#X = pd.get_dummies( XY[ columns ].astype('str'), columns=columnsCAT )
y_ = pd.get_dummies( XY[ target_column ].astype('str') ) 
y = y_[ target_column_onehot ]
print(columns,X.shape,y.shape)
print(XY.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_scaler = skl.preprocessing.StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
print( "Score", balanced_accuracy_score(y_test, y_pred) )
    #break

# save the model to curent directory
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)


['Categorical_BMI', 'Education', 'Poverty_Ratio', 'Race', 'Age', 'Diabetes']
['Categorical_BMI', 'Education', 'Poverty_Ratio', 'Race', 'Age'] (29482, 28) (29482,)
  Categorical_BMI                    Education  Poverty_Ratio  \
0      Overweight                   Grade_1-11           1.93   
1      Overweight  Associates_Academic_Program           4.45   
2      Overweight                     Bachelor           5.94   
3           Obese       Some_College_no_degree           3.70   
4           Obese         High_School_Graduate           1.66   

                    Race  Age Diabetes  
0             White_Only   50       no  
1  African_American_Only   53      yes  
2             White_Only   56       no  
3             White_Only   57       no  
4  African_American_Only   25       no  
Score 0.7073361491248997
0.6757563424230091


/Users/yinglin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
X_test

,Poverty_Ratio,Age,Categorical_BMI_Healthy_Weight,Categorical_BMI_Obese,Categorical_BMI_Overweight,Categorical_BMI_Underweight,Categorical_BMI_Unknown,Education_12th_Grade_no_diploma,Education_Associates_Academic_Program,Education_Associates_Occupational_Technical_Vocational,...,Education_Some_College_no_degree,Race_AIAN_AND_other,Race_AIAN_Only,Race_African_American_Only,Race_Asian_Only,Race_Dont_Know,Race_Not_Ascertained,Race_Other,Race_Refused,Race_White_Only
10675,2.54,85,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14080,3.34,39,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7111,4.85,43,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2101,0.57,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22333,3.19,72,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2592,11.00,34,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4787,3.75,49,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16708,7.03,73,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17847,3.23,43,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [3]:
X_train

,Poverty_Ratio,Age,Categorical_BMI_Healthy_Weight,Categorical_BMI_Obese,Categorical_BMI_Overweight,Categorical_BMI_Underweight,Categorical_BMI_Unknown,Education_12th_Grade_no_diploma,Education_Associates_Academic_Program,Education_Associates_Occupational_Technical_Vocational,...,Education_Some_College_no_degree,Race_AIAN_AND_other,Race_AIAN_Only,Race_African_American_Only,Race_Asian_Only,Race_Dont_Know,Race_Not_Ascertained,Race_Other,Race_Refused,Race_White_Only
23533,0.51,54,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2887,7.42,60,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16510,0.26,57,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18871,1.48,64,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9443,3.79,41,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,2.19,37,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
19029,4.01,74,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9886,1.73,39,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6822,8.64,55,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
# loc. first data pt. 
X_test.iloc[0,:].values

array([ 2.54, 85.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        1.  ])

In [5]:
# place first data pt into list 
XX = [2.54, 85, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1]


In [7]:
col_names = list( X_test.columns.values )
col_names

['Poverty_Ratio',
 'Age',
 'Categorical_BMI_Healthy_Weight',
 'Categorical_BMI_Obese',
 'Categorical_BMI_Overweight',
 'Categorical_BMI_Underweight',
 'Categorical_BMI_Unknown',
 'Education_12th_Grade_no_diploma',
 'Education_Associates_Academic_Program',
 'Education_Associates_Occupational_Technical_Vocational',
 'Education_Bachelor',
 'Education_Dont_Know',
 'Education_GED_Equivalent',
 'Education_Grade_1-11',
 'Education_Greater_Than_Master',
 'Education_High_School_Graduate',
 'Education_Masters',
 'Education_Refused',
 'Education_Some_College_no_degree',
 'Race_AIAN_AND_other',
 'Race_AIAN_Only',
 'Race_African_American_Only',
 'Race_Asian_Only',
 'Race_Dont_Know',
 'Race_Not_Ascertained',
 'Race_Other',
 'Race_Refused',
 'Race_White_Only']

In [8]:
df_XX = pd.DataFrame(data=[dict(zip(col_names, XX) ) ] )
df_XX

,Poverty_Ratio,Age,Categorical_BMI_Healthy_Weight,Categorical_BMI_Obese,Categorical_BMI_Overweight,Categorical_BMI_Underweight,Categorical_BMI_Unknown,Education_12th_Grade_no_diploma,Education_Associates_Academic_Program,Education_Associates_Occupational_Technical_Vocational,...,Education_Some_College_no_degree,Race_AIAN_AND_other,Race_AIAN_Only,Race_African_American_Only,Race_Asian_Only,Race_Dont_Know,Race_Not_Ascertained,Race_Other,Race_Refused,Race_White_Only
0,2.54,85,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
model.predict_proba( df_XX )

array([[0.21521541, 0.78478459]])

In [10]:
model.classes_

array([0, 1], dtype=uint8)

In [11]:
y_test

10675    0
14080    0
7111     0
2101     0
22333    0
        ..
2592     0
4787     0
16708    0
17847    0
13273    0
Name: yes, Length: 7371, dtype: uint8

In [12]:
X_test

,Poverty_Ratio,Age,Categorical_BMI_Healthy_Weight,Categorical_BMI_Obese,Categorical_BMI_Overweight,Categorical_BMI_Underweight,Categorical_BMI_Unknown,Education_12th_Grade_no_diploma,Education_Associates_Academic_Program,Education_Associates_Occupational_Technical_Vocational,...,Education_Some_College_no_degree,Race_AIAN_AND_other,Race_AIAN_Only,Race_African_American_Only,Race_Asian_Only,Race_Dont_Know,Race_Not_Ascertained,Race_Other,Race_Refused,Race_White_Only
10675,2.54,85,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14080,3.34,39,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7111,4.85,43,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2101,0.57,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22333,3.19,72,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2592,11.00,34,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4787,3.75,49,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16708,7.03,73,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17847,3.23,43,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [13]:
df_XX.loc[0,'Race_Dont_Know']

0

In [14]:
df_XX

,Poverty_Ratio,Age,Categorical_BMI_Healthy_Weight,Categorical_BMI_Obese,Categorical_BMI_Overweight,Categorical_BMI_Underweight,Categorical_BMI_Unknown,Education_12th_Grade_no_diploma,Education_Associates_Academic_Program,Education_Associates_Occupational_Technical_Vocational,...,Education_Some_College_no_degree,Race_AIAN_AND_other,Race_AIAN_Only,Race_African_American_Only,Race_Asian_Only,Race_Dont_Know,Race_Not_Ascertained,Race_Other,Race_Refused,Race_White_Only
0,2.54,85,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
